## Downstream task #########

In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')  # ignore warnings, like ZeroDivision

# Go up one directory from 'notebooks/' to project root
project_root = os.path.abspath('..')
sys.path.append(project_root)

In [2]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
    print("GPU is available. Using GPU for computation.")
else:
    # If GPU is not available, fall back to CPU
    device = torch.device("cpu")
    print("GPU is not available. Using CPU for computation.")

GPU is available. Using GPU for computation.


In [ ]:
from src.transformation_utils import *
from src.utils_all import *
from src.utils_data import *

from src.MAE.utils_mae import *
from src.MAE.trainer_mae import *
from src.MAE.MAE_1D import *
from src.MAE.multi_trait import *
from src.MAE.trainer_trait import *

# import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split

import gc

##################
d = 10
h = 16
w_loss = 1

SeqSize = 20
percentage_tr = 1
batch_size = 256

perc = 100

checkpoints_path = 'mae_model/checkpoints_MAE_250415_0421_Training100_allUNlabels_155_New_128/'
checkpoint_dir_mae = os.path.join(project_root, checkpoints_path)

################ Data ###############
path_data_lb = os.path.join(project_root, "Datasets/50_all_traits.csv")

ls_tr = ["cab", "cw", "cm", "LAI", "cp", "cbc", "car", "anth"]



In [ ]:
# #### MAE_FR>> bands:1720, type_sp='full' default
# #### MAE_HR>> bands:500, type_sp='full'
# ### MAE_FR_HR >> bands:1720, type_sp='half'

mean_metrics, std_metrics = run_consistent_experiment(checkpoint_dir_mae, path_data_lb, seeds=[155, 381, 187], fine_tune=False, n_epochs = 200, percentage_tr=1, type_sp='full', n_bands=1720,  save=False, name='')
mean_metrics, std_metrics = run_consistent_experiment(checkpoint_dir_mae, path_data_lb, seeds=[155, 381, 187], fine_tune=True, n_epochs = 80, percentage_tr=1, type_sp='full', n_bands=1720,  save=False, name='')


In [11]:
import torch
test_mae = load_model(checkpoint_dir_mae, type_sp='full', n_bands=1720, seq_size=20,d=10, h=16,mask_ratio=0.75)

# Calculate the number of trainable parameters
num_trainable_params = sum(p.numel() for p in test_mae.model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {num_trainable_params}")

Encoder has 2006912 parameters.
Decoder has 1220116 parameters.
Number of trainable parameters: 3192084


In [ ]:
############## TRansferabiloty ###
eval_scores, ext_scores = run_consistent_experimentCV(checkpoint_dir_mae, [155], fine_tune=False, n_epochs = 200, percentage_tr=1, type_sp='full', n_bands=1720,  save=True,  start=0, end=None)
eval_scores, ext_scores = run_consistent_experimentCV(checkpoint_dir_mae, [155], fine_tune=True, n_epochs = 80, percentage_tr=1, type_sp='full', n_bands=1720,  save=True,  start=0, end=None)

In [6]:
# Concatenate all DataFrames along a new axis and calculate mean and std
all_dfs = pd.concat(eval_scores.values(), keys=eval_scores.keys())  #eval_scores ext_scores

# Compute average and standard deviation across the dictionary
# Compute mean and standard deviation while keeping the original order
mean_metrics = all_dfs.groupby(level=1, sort=False).mean()
std_metrics = all_dfs.groupby(level=1, sort=False).std(ddof=0) ## STDS ddof=1

In [7]:
mean_metrics

,r2_score,RMSE,nRMSE (%),MAE,Bias,RPD
cab,0.517631,9.409538,17.312572,6.691334,0.578276,1.445159
cw,0.391854,0.007004,16.577836,0.004181,0.000281,1.286574
cm,0.397031,0.004919,14.381215,0.002743,0.000156,1.289751
LAI,0.567053,1.162055,17.262958,0.851097,-0.056028,1.523229
cp,0.477985,0.000318,12.561457,0.000193,0.000021,1.391700
cbc,0.401564,0.004937,14.563138,0.002656,0.000034,1.297041
car,0.417586,2.184357,15.415402,1.512781,0.044853,1.314069
anth,0.546895,0.290524,14.973379,0.191235,0.059086,1.505080


In [8]:
std_metrics

,r2_score,RMSE,nRMSE (%),MAE,Bias,RPD
cab,0.038161,0.376368,0.692477,0.147580,0.948763,0.058967
cw,0.045050,0.000262,0.620823,0.000088,0.000203,0.049150
cm,0.028198,0.000115,0.336784,0.000053,0.000497,0.030278
LAI,0.015666,0.021131,0.313906,0.037679,0.096996,0.027967
cp,0.056942,0.000018,0.691611,0.000004,0.000032,0.077800
cbc,0.048197,0.000200,0.589085,0.000034,0.000585,0.052839
car,0.034107,0.064186,0.452972,0.031854,0.065821,0.038848
anth,0.056314,0.018385,0.947529,0.007719,0.002688,0.098202
